In [29]:
import os, sys

import pandas as pd
import numpy as np

import io

from docx import Document
from docx.shared import Inches, Pt
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

import PIL
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [13]:
# Getting the quantitative data
dir_excel_quanti = 'D:\\SOLSA\\Analysis_quanti\\LAB analysis compilation_XRF-XRD_SLN powders & cores sample.xlsx'

df_quanti_all = pd.read_excel(dir_excel_quanti,sheet_name='XRD_plot_10 Cores')

# Get the sample names
col = df_quanti_all.columns

names_quant = []
for c in col:
    if 'ER-NC' in c:
        names_quant = names_quant + [c.split('_')[0]]

dict_df_quanti = {}
for i,name in enumerate(names_quant):
    dict_df_quanti[name] = df_quanti_all.iloc[:,i*2:i*2+2]
    dict_df_quanti[name].columns = ['Minerals','Prop']
    dict_df_quanti[name] = dict_df_quanti[name].dropna()

names_quant

['ER-NC00-0046',
 'ER-NC00-0047',
 'ER-NC00-0053',
 'ER-NC00-0054',
 'ER-NC00-0055',
 'ER-NC00-0056',
 'ER-NC00-0060',
 'ER-NC00-0061',
 'ER-NC00-0063',
 'ER-NC00-0073']

In [14]:
dir_rap = 'D:\\SOLSA\\DATA\\111_fused\\Ech_ref\\_rapport_samples'





fig_names = ['rgb',
             'clustering_table_minerals',
             'clustering_minerals_ind',
             'custom_tree_table_minerals',
             'custom_tree_minerals_ind',
             'dist_table_minerals',
             'dist_minerals_ind',
             'lib_tree_table_minerals',
             'lib_tree_minerals_ind',
             'unmixing_table_minerals',
             'unmixing_minerals_ind',
             ]

exts = ['.png','.csv']

dict_fig = {}
names = []

file_list = os.listdir(dir_rap)

for f in file_list:
    if 'times' in f:
        pass
    else:
        # Get the names
        name = f
        for sub in fig_names:
            if sub in f:
                name = name.replace('_' + sub,'')
                fig_name = sub

        for sub in exts:
            name = name.replace(sub,'')

        # Keep unique values
        names = list(set(names + [name]))
        names.sort()

        # Fill the data dictionary
        # init dictionnary
        try:
            dict_tmp = dict_fig[name]
        except:
            dict_tmp = {}

        path_file = os.path.join(dir_rap,f)
        if '.csv' in f:
            elt = pd.read_csv(path_file)
        elif '.png' in f:
            elt = Image.open(path_file)

        dict_fig[name] = {**dict_tmp, **{fig_name:elt}}

for name in names:
    if name in names_quant:
        dict_fig[name] = {**dict_fig[name], **{'tab_quanti':dict_df_quanti[name]}}
    else:
        dict_fig[name] = {**dict_fig[name], **{'tab_quanti':''}}
        
print(names)

['11MC_harz', '11MC_kaolinite', '11MC_sap_terreuse', 'BST_1', 'ER-NC00-0046', 'ER-NC00-0046-53-54_slice_Anas', 'ER-NC00-0047', 'ER-NC00-0048', 'ER-NC00-0049', 'ER-NC00-0051', 'ER-NC00-0053', 'ER-NC00-0054', 'ER-NC00-0055', 'ER-NC00-0055-56-60_slice_Anas', 'ER-NC00-0056', 'ER-NC00-0057', 'ER-NC00-0059', 'ER-NC00-0060', 'ER-NC00-0061', 'ER-NC00-0063', 'ER-NC00-0063-73_slice_Anas', 'ER-NC00-0064', 'ER-NC00-0067', 'ER-NC00-0068', 'ER-NC00-0070', 'ER-NC00-0071', 'ER-NC00-0072', 'ER-NC00-0073', 'GR', 'HN5', 'LJ_2', 'LR']


In [26]:
df = pd.DataFrame([[np.nan,np.nan],[np.nan,np.nan]])

df.columns = ['a','b']

sum(~df['a'].isna())

0

In [68]:
def add_table_from_df(doc,df_in):
    df = df_in.copy()
    t = doc.add_table(df.shape[0] + 1, df.shape[1])
    
    try:
        color = get_hex_color(df)
        df['Color'] = ''
    except:
        pass
    
    # add the header rows
    for j in range(df.shape[-1]):
        t.cell(0,j).text = df.columns[j]
        paragraph = t.cell(0,j).paragraphs[0]
        run = paragraph.runs
        font = run[0].font
        font.size = Pt(9)
        """p_element = paragraph._element
        print(dir(paragraph.paragraph_format))
        print(paragraph.paragraph_format._line_spacing_rule)
        print(paragraph.paragraph_format.keep_together)
        print(paragraph.paragraph_format.space_before)
        print(p_element.xpath(r'w:pPr/w:contextualSpacing'))"""
        
        """cspacing = p_element.xpath(r'w:pPr/w:contextualSpacing')[0]
        cspacing.getparent().remove(cspacing)"""
    
    # add the rest of the data frame
    for i in range(df.shape[0]):
        for j in range(df.shape[-1]):
            t.cell(i+1,j).text = str(df.values[i,j])
            if df.columns[j] == 'Color':
                shading_elm_1 = parse_xml((r'<w:shd {} w:fill="' + color[i] + '"/>').format(nsdecls('w')))
                t.cell(i+1,j)._tc.get_or_add_tcPr().append(shading_elm_1)
                t.cell(i+1,j).text = ' '
            paragraph = t.cell(i+1,j).paragraphs[0]
            run = paragraph.runs
            font = run[0].font
            font.size = Pt(9)
            
            """p_element = paragraph._element
            cspacing = p_element.xpath(r'w:pPr/w:contextualSpacing')[0]
            cspacing.getparent().remove(cspacing)"""
            
def get_hex_color(df):
    
    return df['Color'].str.replace('<html><table border=0 width=400 bgcolor=#','').str.replace('><TR><TD> </TD></TR></table>','')
    
            
def add_img_to_table(tab,img):
    paragraph = tab.paragraphs[0]
    run = paragraph.add_run()
    run.add_picture(image2file(img), width = 2800000, height = 2800000)

def image2file(image):
    """Return `image` as PNG file-like object."""
    image_file = io.BytesIO()
    image.save(image_file, format="PNG")
    return image_file

In [75]:
print(np.gradient([10,15,17,20],[1,2,3,4]))
print(np.gradient([10,15,17,20],1))

[5.  3.5 2.5 3. ]
[5.  3.5 2.5 3. ]


In [72]:
dict = {'dd':'d'}

dict.values()

dict_values(['d'])

In [70]:
doc = Document()

doc.add_heading('Tests methods samples', 0)

for name in names:
    doc.add_heading(name, 1)
    print(name)
    table = doc.add_table(rows=len(dict_fig[name]), cols=2, style="Table Grid")
    
    for ifig,fig in enumerate(['tab_quanti'] + fig_names):
        fig_tmp = dict_fig[name][fig]
        
        col = ifig % 2
        row_title = int(np.floor(ifig/2)*2)
        row_fig = row_title + 1
        
        if isinstance(fig_tmp,pd.DataFrame):           
            table.cell(row_title,col).text = fig
            add_table_from_df(table.cell(row_fig,col),fig_tmp)
            
        elif isinstance(fig_tmp,PIL.PngImagePlugin.PngImageFile):
            table.cell(row_title,col).text = fig
            add_img_to_table(table.cell(row_fig,col),fig_tmp)
            
        elif isinstance(fig_tmp,str):
            table.cell(row_title,col).text = fig
            table.cell(row_fig,col).text = fig_tmp

"""for p in doc.paragraphs:
    p_element = p._element
    cspacing = p_element.xpath(r'w:pPr/w:contextualSpacing')[0]
    cspacing.getparent().remove(cspacing)"""
            
doc.save('demo.docx')

11MC_harz
11MC_kaolinite
11MC_sap_terreuse
BST_1
ER-NC00-0046
ER-NC00-0046-53-54_slice_Anas
ER-NC00-0047
ER-NC00-0048
ER-NC00-0049
ER-NC00-0051
ER-NC00-0053
ER-NC00-0054
ER-NC00-0055
ER-NC00-0055-56-60_slice_Anas
ER-NC00-0056
ER-NC00-0057
ER-NC00-0059
ER-NC00-0060
ER-NC00-0061
ER-NC00-0063
ER-NC00-0063-73_slice_Anas
ER-NC00-0064
ER-NC00-0067
ER-NC00-0068
ER-NC00-0070
ER-NC00-0071
ER-NC00-0072
ER-NC00-0073
GR
HN5
LJ_2
LR


In [148]:
fig_tmp = dict_fig[names[0]]['clustering_minerals_ind']
isinstance(fig_tmp,PIL.PngImagePlugin.PngImageFile)

print(fig_names)

['rgb', 'clustering_minerals_ind', 'clustering_table_minerals', 'custom_tree_minerals_ind', 'custom_tree_table_minerals', 'dist_minerals_ind', 'dist_table_minerals', 'lib_tree_minerals_ind', 'lib_tree_table_minerals', 'unmixing_minerals_ind', 'unmixing_table_minerals']


In [119]:
pic = "some_picture.jpg"
document = Document()
tbl = document.add_table(rows=3, cols=2, style="Table Grid")
paragraph = tbl.cell(1,1).paragraphs[0]
run = paragraph.add_run()
run.add_picture(image2file(fig_tmp), width = 140000, height = 140000)
document.save("demo.docx")